In [1]:
from pymetamap import MetaMap
from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm_notebook
import pickle

In [2]:
mm = MetaMap.get_instance('/home/bsingh/public_mm/bin/metamap16') 

In [3]:
# Check the metamap instance is working.
mm.extract_concepts(['Heart Attack', 'I am having a heart attack',], [0,1])

([ConceptMMI(index='0', mm='MMI', score='14.64', preferred_name='Myocardial Infarction', cui='C0027051', semtypes='[dsyn]', trigger='["-- Heart Attack"-tx-1-"Heart Attack"-noun-0]', location='TX', pos_info='1/12', tree_codes='C14.280.647.500;C14.907.585.500'),
  ConceptMMI(index='1', mm='MMI', score='17.80', preferred_name='Iodides', cui='C0021966', semtypes='[inch]', trigger='["I-"-tx-1-"I"-noun-0]', location='TX', pos_info='1/1', tree_codes='D01.248.497.158.490;D01.475.410'),
  ConceptMMI(index='1', mm='MMI', score='14.64', preferred_name='Myocardial Infarction', cui='C0027051', semtypes='[dsyn]', trigger='["-- Heart Attack"-tx-1-"heart attack"-noun-0]', location='TX', pos_info='15/12', tree_codes='C14.280.647.500;C14.907.585.500'),
  ConceptMMI(index='1', mm='MMI', score='5.18', preferred_name='Blood group antibody I', cui='C0221138', semtypes='[aapp,imft]', trigger='["I NOS"-tx-1-"I"-noun-0]', location='TX', pos_info='1/1', tree_codes='')],
 None)

## Working functions
------------------------

In [4]:
def get_semantic_types_dict(list_, curr_list_concepts, sel_sem_types):
    dict_con = {}
    failed_con = 0
    for ind in range(len(list_)):
        dict_con[ind] = []
    for each_con in tqdm(curr_list_concepts):
        try:
            pos_info = each_con.pos_info
            pos_info = pos_info.replace('[','')
            pos_info = pos_info.replace(']','')
            if(';' in pos_info):
                all_pos = pos_info.split(';')
            else:
                all_pos = pos_info.split(',')
            all_pos_final = []
            for each_pos in all_pos:
                all_pos_final+=each_pos.split(',')
            for each_pos in all_pos_final:
                st = int(each_pos.split('/')[0])-1
                end = st+int(each_pos.split('/')[1])
                semtype_all = each_con.semtypes[1:-1].split(',')
                semtype_all = [x for x in semtype_all if(x in sel_sem_types)]
                # can be handled better
                if(len(semtype_all)>0):
                    semtype = semtype_all[0]
                else:
                    semtype = 'Ignore'
                if(semtype in sel_sem_types):
                    dict_con[int(each_con.index)].append([semtype, st, end, 1.0])
        except:
            failed_con+=1
    print("Total_concepts", curr_list_concepts.__len__())
    print("Failed_concepts", failed_con)
    final_dict = {}
    for ind in range(len(list_)):
        final_dict[list_[ind]] = dict_con[ind]
    return final_dict

In [35]:
def get_all_concepts(curr_mm, all_sents, batch_size=10):
    all_concepts = []
    for i in tqdm_notebook(range(0, all_sents.__len__(), batch_size)):
        try:
            curr_sents = all_sents[i:i+batch_size]
            tmp = curr_mm.extract_concepts(curr_sents, [x for x in range(i, i+curr_sents.__len__())])
            all_concepts.append(tmp)
        except: 
            print("problem with the batch starting from"+str(i))
    return all_concepts

In [36]:
def collate_all_concepts(tmp_concepts):
    final_concepts = []
    for each_concept in tmp_concepts:
        final_concepts+=each_concept[0]
    return final_concepts

In [36]:
data = pickle.load(open('../data/emrqa_parawise_data.pkl','rb'))

In [37]:
# As we need the semantic types for the questions as well as paras from both splits
all_data = data['strict_split']['train']+data['strict_split']['dev']+data['strict_split']['test']
all_data = all_data+data['split']['train']+data['split']['dev']+data['split']['test']

In [38]:
# Here the evidence_sentence refers to the paragraphs extracted earlier.
all_sents = [x['evidence_sentence'] for x in all_data]
all_ques = [x['question'] for x in all_data]

In [39]:
all_sents = list(set(all_sents))
all_ques = list(set(all_ques))

In [48]:
selected_sem_types = open('../data/selected_semantic_types.txt','r').read()

selected_sem_types = selected_sem_types.split('\n')
selected_sem_types = [x.split(':')[0].strip() for x in selected_sem_types]

In [51]:
# chosen semantic types
selected_sem_types

['acab',
 'aggp',
 'anab',
 'anst',
 'bpoc',
 'cgab',
 'clnd',
 'diap',
 'emod',
 'evnt',
 'fndg',
 'inpo',
 'lbpr',
 'lbtr',
 'phob',
 'qlco',
 'qnco',
 'sbst',
 'sosy',
 'topp']

In [86]:
all_emrqa_sents_con = get_all_concepts(all_sents=all_sents)

In [87]:
all_emrqa_ques_con = get_all_concepts(all_sents=all_ques)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [123]:
final_emrqa_sents_con = collate_all_concepts(all_emrqa_sents_con)

In [187]:
dict_con_new_sent = get_semantic_types_dict(all_sents, final_emrqa_sents_con, selected_sem_types)

Total_concepts 152378
Failed_concepts 5


In [164]:
final_emrqa_ques_con = collate_all_concepts(all_emrqa_ques_con)

In [188]:
dict_con_new_ques = get_semantic_types_dict(all_ques, final_emrqa_ques_con, selected_sem_types)

Total_concepts 759661
Failed_concepts 17


## Dumping the metamap annotations
---------------------

In [189]:
pickle.dump((dict_con_new_ques, dict_con_new_sent), 
            open('../data/metamap_annotations_emrqa_dict.pkl','wb'))

In [190]:
pickle.dump(selected_sem_types, open('../data/selected_ents.pkl','wb'))

## Updating the data with the semantic type infomrmation
----------------------------------

In [ ]:
concept_dict = pickle.load(open('../data/metamap_annotations_emrqa_dict_all.pkl', 'rb'))

In [ ]:
ques_concept_dict = concept_dict[0]
sent_concept_dict = concept_dict[1]

In [ ]:
def add_entity_information(list_, dict_ques, dict_sents):
    for each_ in tqdm(list_):
        each_['sent_ents'] = dict_sents[each_['evidence_sentence']]
        each_['ques_ents'] = dict_ques[each_['question']]
    print('Done.')

In [ ]:
add_entity_information(data['split']['train'], ques_concept_dict, sent_concept_dict)

In [ ]:
print('---'*10)
print('Updated Datapoint Example')
print('---'*10)
print(data['split']['train'][0])

In [ ]:
add_entity_information(data['split']['dev'], ques_concept_dict, sent_concept_dict)
add_entity_information(data['split']['test'], ques_concept_dict, sent_concept_dict)
##################### STRICT #####################
add_entity_information(data['strict_split']['train'], ques_concept_dict, sent_concept_dict)
add_entity_information(data['strict_split']['dev'], ques_concept_dict, sent_concept_dict)
add_entity_information(data['strict_split']['test'], ques_concept_dict, sent_concept_dict)

In [ ]:
pickle.dump(data, open('../data/emrqa_parawise_data_w_entities.pkl', 'wb'))